In [ ]:
import warnings
warnings.filterwarnings('ignore')  # Ignora avisos para manter o output limpo

from crewai import Agent, Task, Crew
import os
from utils import get_openai_api_key  # Importa a função para obter a chave da API do OpenAI

# Obtém a chave da API do OpenAI
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'  # Define o modelo a ser usado

# Criação do agente de suporte
support_agent = Agent(
    role="Senior Support Representative",  # Define o papel do agente
    goal="Ser o representante de suporte mais amigável e prestativo da equipe",  # Objetivo do agente
    backstory=(
        "Você trabalha na crewAI (https://crewai.com) e "
        "está agora fornecendo suporte para {customer}, um cliente super importante "
        "para sua empresa. "
        "Você precisa garantir que forneça o melhor suporte! "
        "Certifique-se de fornecer respostas completas e não fazer suposições."
    ),
    allow_delegation=False,  # Não permite delegação de tarefas
    verbose=True  # Ativa o modo verboso para mais detalhes
)

# Criação do agente de garantia de qualidade de suporte
support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",  # Define o papel do agente
    goal="Get recognition for providing the "
         "best support quality assurance in your team",  # Objetivo do agente
    backstory=(
        "Você trabalha na crewAI (https://crewai.com) e "
        "está agora trabalhando com sua equipe "
        "em um pedido de {customer}, garantindo que "
        "o representante de suporte esteja fornecendo o melhor suporte possível.\n"
        "Você precisa garantir que o representante de suporte "
        "esteja fornecendo respostas completas e não faça suposições."
    ),
    verbose=True  # Ativa o modo verboso
)

# Importa ferramentas necessárias para busca e raspagem
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

# Criação da ferramenta de raspagem de website
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"  # URL da documentação
)

# Definição da tarefa de resolução de consultas
inquiry_resolution = Task(
    description=(
        "{customer} acabou de entrar em contato com um pedido super importante:\n"
        "{inquiry}\n\n"
        "{person} de {customer} é quem entrou em contato. "
        "Certifique-se de usar tudo o que você sabe "
        "para fornecer o melhor suporte possível."
        "Você deve se esforçar para fornecer uma resposta completa "
        "e precisa à consulta do cliente."
    ),
    expected_output=(
        "Uma resposta detalhada e informativa à "
        "consulta do cliente que aborda "
        "todos os aspectos da pergunta deles.\n"
        "A resposta deve incluir referências "
        "a tudo que você usou para encontrar a resposta, "
        "incluindo dados ou soluções externas. "
        "Certifique-se de que a resposta seja completa, "
        "sem deixar perguntas sem resposta, e mantenha um tom amigável e prestativo."
    ),
    tools=[docs_scrape_tool],  # Ferramenta que o agente pode usar
    agent=support_agent,  # Agente responsável pela tarefa
)

# Definição da tarefa de revisão de qualidade
quality_assurance_review = Task(
    description=(
        "Revise a resposta elaborada pelo Representante de Suporte Sênior para a consulta de {customer}. "
        "Certifique-se de que a resposta seja abrangente, precisa e atenda aos "
        "altos padrões de qualidade esperados para o suporte ao cliente.\n"
        "Verifique se todas as partes da consulta do cliente "
        "foram abordadas "
        "de forma completa, com um tom amigável e prestativo.\n"
        "Verifique as referências e fontes usadas para "
        "encontrar as informações, "
        "garantindo que a resposta seja bem fundamentada e "
        "não deixe perguntas sem resposta."
    ),
    expected_output=(
        "Uma resposta final, detalhada e informativa "
        "pronta para ser enviada ao cliente.\n"
        "Esta resposta deve abordar completamente a "
        "consulta do cliente, incorporando todo "
        "o feedback e melhorias relevantes.\n"
        "Não seja muito formal, somos uma empresa descontraída e legal "
        "mas mantenha um tom profissional e amigável durante todo o processo."
    ),
    agent=support_quality_assurance_agent,  # Agente responsável pela tarefa
)

# Criação da equipe com os agentes e tarefas
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],  # Lista de agentes
    tasks=[inquiry_resolution, quality_assurance_review],  # Lista de tarefas
    verbose=2,  # Nível de verbosidade
    memory=True  # Ativa a memória para o crew
)

# Definição dos inputs para a tarefa
inputs = {
    "customer": "DeepLearningAI",  # Nome do cliente
    "person": "Andrew Ng",  # Nome da pessoa que entrou em contato
    "inquiry": "Preciso de ajuda para configurar uma Crew "
               "e iniciá-la, especificamente "
               "como posso adicionar memória à minha crew? "
               "Você pode fornecer orientações?"
}

# Inicia a crew com os inputs definidos
result = crew.kickoff(inputs=inputs)

# Exibe o resultado da execução
from IPython.display import Markdown
Markdown(result)  # Mostra a resposta gerada

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI acabou de entrar em contato com um pedido super importante:
Preciso de ajuda para configurar uma Crew e iniciá-la, especificamente como posso adicionar memória à minha crew? Você pode fornecer orientações?

Andrew Ng de DeepLearningAI é quem entrou em contato. Certifique-se de usar tudo o que você sabe para fornecer o melhor suporte possível.Você deve se esforçar para fornecer uma resposta completa e precisa à consulta do cliente.


> Entering new CrewAgentExecutor chain...
Eu devo garantir que forneça uma resposta completa e precisa para o cliente, utilizando todas as ferramentas disponíveis para mim.

Action: Read website content
Action Input: {} 

Introduction - CrewAICrewAI home pageSearch CrewAI docsStart Free TrialcrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationGet StartedIntroductionDocumentationEnterpriseExamplesReleasesWebsiteForumGet HelpGet StartedIntroductionInstallationQu

Final Answer: 
Para adicionar memória à sua crew no CrewAI, você pode seguir as seguintes etapas:

1. Crie uma Crew: O primeiro passo é criar uma equipe no CrewAI. Você pode fazer isso seguindo o tutorial passo a passo para criar sua primeira equipe colaborativa de IA que trabalha junta para resolver problemas complexos. Você pode encontrar o tutorial em https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/.

2. Adicione Agentes: Após criar a sua equipe, você precisa adicionar agentes com papéis especializados, conhecimento e objetivos definidos. Os agentes são os membros da sua equipe e desempenham funções específicas, como pesquisador, escritor, entre outros. Certifique-se de equipar os agentes com ferramentas personalizadas e APIs para interagir com serviços externos e fontes de dados.

3. Defina Tarefas: Uma vez que seus agentes tenham sido adicionados à equipe, defina tarefas individuais com objetivos claros, ferramentas específicas e resultados acionáveis. As tarefas sã